In [250]:
import functools
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from queue import PriorityQueue
from numpy import array_str
import heapq
import logging




In [204]:
PUZZLE_DIM = 2  ### change to 5
action = namedtuple('Action', ['pos1', 'pos2'])

In [39]:
### put before the function the @
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper

In [ ]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

More function

In [205]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
goal = state
print(state)

[[1 2]
 [3 0]]


In [207]:
def goal_function( state ,goal ):
    label = np.array_equal(state, goal)
    if(label):
        return True
    else:
        return False

In [208]:
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 150450.28it/s]


array([[0, 3],
       [2, 1]])

In [ ]:
class CustomPriorityQueue:
    def __init__(self):
        self.pq = PriorityQueue()  # The actual PriorityQueue
        self.items_set = set()  # Set to track items in the queue
        self.count = 0  # Variable to keep track of the number of elements
    
    def put(self, item, priority):
        self.pq.put((item, priority))  # Add to the PriorityQueue
        self.items_set.add(priority)  # Track item in the set
        self.count += 1  # Increment the count when an item is added

    def get(self):
        priority, item = self.pq.get()  # Get from the PriorityQueue
        self.items_set.remove(priority)  # Remove from the set
        self.count -= 1  # Increment the count when an item is added
        return item, priority
    
    def contains(self, item):
        return item in self.items_set  # Check if the item exists in the set
    
    def empty(self):
        return self.pq.empty()  # Check if the PriorityQueue is empty
    
    def size(self):
        return self.count  # Return the number of elements in the queue

In [248]:
####

def search( initial_state , cost_function):
    state =initial_state
    frontier = CustomPriorityQueue()
    state_cost = dict() 
    parent_state = dict()
    global_state = dict() 

    state_cost[array_str(state)] = 0
    parent_state[array_str(state)] = None
    global_state[array_str(state)] = state

    while state is not None and not goal_function(state, goal):
        for a in available_actions(state):
            new_state = do_action(state,a)
            cost = cost_function(a)   ### given an action, give you the cost
            if(array_str(new_state) not in state_cost and not frontier.contains(array_str(new_state)) ):
                state_cost[array_str(new_state)]= state_cost[array_str(state)]  + cost
                new_cost = state_cost[array_str(new_state)]
                parent_state[array_str(new_state)] = state
                global_state[array_str(new_state)] = new_state
                frontier.put(  new_cost  , array_str(new_state)   )  ### priority item
            elif( frontier.contains(array_str(new_state)) and state_cost[array_str(new_state)] > state_cost[array_str(state)] + cost):  #array_str(new_state) in frontier
                parent_state[array_str(new_state)] = state
                state_cost[array_str(new_state)] = state_cost[array_str(state)] + cost
                global_state[array_str(new_state)] = new_state
        
        if not frontier.empty():
                tuple = frontier.get()
                arr_state = tuple[0]
                state = global_state[arr_state]
        else:
            state = None
    path = list()
    s = state
    flag =True
    while flag:        
        # Check if array_str(s) is not None
        if array_str(s) is not None:
            path.append(parent_state[array_str(s)])
            s = parent_state[array_str(s)]
            if(s is None):
                flag= False
    print(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
    return list(reversed(path))


In [249]:
solution = search( state , lambda x: 1)
print(solution)

Found a solution in 7 steps; visited 12 states
[None, array([[0, 3],
       [2, 1]]), array([[3, 0],
       [2, 1]]), array([[3, 1],
       [2, 0]]), array([[3, 1],
       [0, 2]]), array([[0, 1],
       [3, 2]]), array([[1, 0],
       [3, 2]])]
